In [ ]:
%matplotlib inline

In [ ]:
from glob import glob

caseid = "NG_5120x2560x34_4km_10s_QOBS_EQX_1280"
step = "0000864000"

stagger_path = f"tmp/stagger/{caseid}/{step}.nc"
center_path = "/home/disk/eos4/nbren12/eos8/Data/id/2/NG_5120x2560x34_4km_10s_QOBS_EQX/coarse/3d/all.nc"
stat_path = "/home/disk/eos4/nbren12/eos8/Data/id/2/NG_5120x2560x34_4km_10s_QOBS_EQX/stat.nc"

stag = xr.open_dataset(stagger_path)
cent = xr.open_dataset(center_path, engine='netcdf4').sel(time=stag.time)
stat = xr.open_dataset(stat_path)

In [ ]:
stag.V[0,:, 0, :].plot()

In [ ]:
stag.V[0,:, -1, :].plot()

Ok, the southern boundary in this dataset is 0. Let's combine the initial conditions

we need to rename the staggered coordinate variables to work with SAM netCDF input

In [ ]:
def rename_var(z, coords):
    rename_d = {'xc': 'x', 'yc': 'y', 'ys': 'y', 'xs': 'x'}
    rename_d = {key: val for key, val in rename_d.items()
                if key in z.dims}
    
    
    return z.rename(rename_d).assign_coords(**coords)
    

stag = stag.apply(lambda x: rename_var(x, cent.coords))

In [ ]:
ic = xr.Dataset({
    'U': stag.U,
    'V': stag.V,
    'W': cent.W,
    'QV': cent.QV,
    'TABS': cent.TABS,
    'QN': cent.QN,
    'QP': cent.QP,
    'RHO' : stat.RHO[0],
    'Ps': stat.Ps[0]
})

ic= ic.isel(time=0)
ic.to_netcdf("ic.nc")

In [ ]:
ic